# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

### Split:
Separate it into a training (70%) and testing set(30%).

### Scale the Feature Data

Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.

In [2]:
def preproc(df,test_size,seedval):
    y_val = df['medianHouseValue']
    x_data = df.drop(['medianHouseValue'],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,
                                        test_size=test_size,random_state=seedval)
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    df_train = pd.DataFrame(data=scaler.transform(X_train),
                           columns = X_train.columns,index=X_train.index)
    df_test = pd.DataFrame(data=scaler.transform(X_test),
                          columns = X_test.columns,index=X_test.index)
    return df_train, y_train, df_test, y_test

In [3]:
def mkInputFunctions(df_train, y_train, df_test, epochs, batches):
    input_func_train = tf.estimator.inputs.pandas_input_fn(
        x=df_train, y=y_train, 
        batch_size=batches,
        num_epochs=epochs,
        shuffle=True
    )
    input_func_test = tf.estimator.inputs.pandas_input_fn(
        x=df_test, 
        batch_size=batches,
        num_epochs=epochs,
        shuffle=False
    )
    return input_func_train, input_func_test

** Import the cal_housing_clean.csv file with pandas. **

In [4]:
df = pd.read_csv('cal_housing_clean.csv')
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


## Do preprocessing setup

In [5]:
test_size, seedval = 0.30, 101
df_train, y_train, df_test, y_test = preproc(df,test_size,seedval)

In [6]:
print( len(df_train['housingMedianAge']), 
len(y_train) )

14448 14448


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [7]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

feat_cols = [ age,rooms,bedrooms,pop,households,income]

In [8]:
df_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [109]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[12,12,12,12,12,12],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_model_dir': '/var/folders/9y/n0rxh9rj60q4v8zgsclp0gm97tm33p/T/tmpjnw79w2u', '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_tf_random_seed': 1}


In [110]:
epochs, batches = 1,10
input_func_train, input_func_test = mkInputFunctions(df_train, y_train, 
                                                     df_test, epochs, batches)

##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [111]:
dnn_model.train(input_fn=input_func_train,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/9y/n0rxh9rj60q4v8zgsclp0gm97tm33p/T/tmpjnw79w2u/model.ckpt.
INFO:tensorflow:loss = 9.54469e+11, step = 1
INFO:tensorflow:global_step/sec: 720.81
INFO:tensorflow:loss = 1.98096e+11, step = 101 (0.139 sec)
INFO:tensorflow:global_step/sec: 732.928
INFO:tensorflow:loss = 6.81793e+10, step = 201 (0.136 sec)
INFO:tensorflow:global_step/sec: 694.691
INFO:tensorflow:loss = 5.19934e+10, step = 301 (0.144 sec)
INFO:tensorflow:global_step/sec: 659.061
INFO:tensorflow:loss = 9.86859e+10, step = 401 (0.151 sec)
INFO:tensorflow:global_step/sec: 776.729
INFO:tensorflow:loss = 6.64853e+10, step = 501 (0.129 sec)
INFO:tensorflow:global_step/sec: 716.491
INFO:tensorflow:loss = 6.97715e+10, step = 601 (0.139 sec)
INFO:tensorflow:global_step/sec: 699.982
INFO:tensorflow:loss = 7.50568e+10, step = 701 (0.143 sec)
INFO:tensorflow:global_step/sec: 683.246
INFO:tensorflow:loss = 5.96176e+10, step = 801 (0.14

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [112]:
predictions = list( dnn_model.predict(input_func_test) )

INFO:tensorflow:Restoring parameters from /var/folders/9y/n0rxh9rj60q4v8zgsclp0gm97tm33p/T/tmpjnw79w2u/model.ckpt-1445


In [113]:
len(predictions)

6192

In [114]:
results = []
for p in predictions:
    results.append(p['predictions'][0])

In [115]:
len(results)

6192

In [116]:
len(y_test)

6192

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [117]:
y_true, y_obs = y_test, results
se = mean_squared_error(y_true, y_obs)

In [118]:
se**0.5

82474.243585167278

# Great Job!